# Yelp API Restaurant Calls By Zip

- Objective: Determine total count of search results of restaurants for each zip
- Purpose: Estimate total number of unique restaurants in the Bay Area
- Result: Stores zip codes and results counts for each in total_count_zip.csv

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from cities_by_zip.csv
- Build function that calls and writes restaurant data to new csv, given a zip
- Run list of zips through function

In [7]:
# Create dataframe from csv file
zip_df = pd.read_csv("Resources/cities_by_zip.csv", names=["Zip", "City", "County"])
zip_df

,Zip,City,County
0,94501,Alameda,Alameda
1,94502,Alameda,Alameda
2,94706,Albany,Alameda
3,94707,Albany,Alameda
4,94710,Albany,Alameda
5,94701,Berkeley,Alameda
6,94702,Berkeley,Alameda
7,94703,Berkeley,Alameda
8,94704,Berkeley,Alameda
9,94705,Berkeley,Alameda


In [14]:
# Function returns 50 restaurant listings for input zip.
# Unlike other API call functions in repo, this is seeking total counts in the area instead of restaurant data
def get_restaurants(zipcode, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0

    # Set parameters and pass into API calls
    params = {"term": "restaurants", "location":zipcode, "limit":50}
    req = requests.get(url, params=params, headers=headers)

    # Breaks if error occurs with search
    if req.status_code == 400:
        count+=1
        print(f"Error for coordinate {count}")
    elif req.status_code == 200:

        # Convert to json
        response = req.json()

        # Log history
        count += 1
        print(f"Now processing query {zipcode}")

        # Iterate through business results and extract data
        for biz in response["businesses"]:

            # Logic to replace missing price level data with NaN
            if "price" not in biz:
                # If also missing category data, replace entry with NaN
                if biz["categories"] == []:
                    restaurant_data.append([zipcode, biz["name"], biz["coordinates"]["latitude"], 
                                        biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                        biz["location"]["zip_code"], biz["rating"], 
                                        biz["review_count"], "", "", biz["id"]])
                else:
                    restaurant_data.append([zipcode, biz["name"], biz["coordinates"]["latitude"], 
                                            biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                            biz["location"]["zip_code"], biz["rating"], 
                                            biz["review_count"], "", biz["categories"][0]["title"], 
                                            biz["id"]])

            # Replace missing category data with empty string
            elif biz["categories"] == []:
                restaurant_data.append([zipcode, biz["name"], biz["coordinates"]["latitude"], 
                                        biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                        biz["location"]["zip_code"], biz["rating"], 
                                        biz["review_count"], biz["price"], "", biz["id"]])

            else:
                restaurant_data.append([zipcode, biz["name"], biz["coordinates"]["latitude"], 
                                        biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                        biz["location"]["zip_code"], biz["rating"], 
                                        biz["review_count"], biz["price"], 
                                        biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/zipcode_datatotestreliability.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()
    
    # Returns total count of restaurants in zip
    return response["total"]


In [15]:
# Track # of zips processed
count = 0

# List to track total restaurants found in zip
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of zips in zip_df
for zipcode in zip_df["Zip"]:
    
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(zipcode, api_key))
    
    # Print log history
    count += 1
    rem = len(zip_df["Zip"]) - count
    print("-----------------------------------------")
    if zipcode == zip_df.iloc[-1,0]:
        print("Full list of zip codes processed!")
    elif zipcode == zip_df.iloc[-2,0]:
        print("Now getting results for final zip code. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for {zipcode} complete")
        print(f"Getting results for next zip code.. there are {rem} zips left")
        print("-----------------------------------------")

zip_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20 for 94501
-----------------------------------------
Data retrieval for 94501 complete
Getting results for next zip code.. there are 394 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94502
-----------------------------------------
Data retrieval for 94502 complete
Getting results for next zip code.. there are 393 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94706
-----------------------------------------
Data retrieval for 94706 complete
Getting results for next zip code.. there are 392 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94707
-----------------------------------------
Data retrieval for 94707 complete
Getting results for next zip code.. there are 391 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94710

Now processing query set 1 of max 20 for 94601
-----------------------------------------
Data retrieval for 94601 complete
Getting results for next zip code.. there are 357 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94602
-----------------------------------------
Data retrieval for 94602 complete
Getting results for next zip code.. there are 356 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94603
-----------------------------------------
Data retrieval for 94603 complete
Getting results for next zip code.. there are 355 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94604
-----------------------------------------
Data retrieval for 94604 complete
Getting results for next zip code.. there are 354 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94605
-----------------------------------------
Data retrie

Now processing query set 1 of max 20 for 94577
-----------------------------------------
Data retrieval for 94577 complete
Getting results for next zip code.. there are 320 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94578
-----------------------------------------
Data retrieval for 94578 complete
Getting results for next zip code.. there are 319 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94579
-----------------------------------------
Data retrieval for 94579 complete
Getting results for next zip code.. there are 318 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94587
-----------------------------------------
Data retrieval for 94587 complete
Getting results for next zip code.. there are 317 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94509
-----------------------------------------
Data retrie

Now processing query set 1 of max 20 for 94807
-----------------------------------------
Data retrieval for 94807 complete
Getting results for next zip code.. there are 283 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94808
-----------------------------------------
Data retrieval for 94808 complete
Getting results for next zip code.. there are 282 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94820
-----------------------------------------
Data retrieval for 94820 complete
Getting results for next zip code.. there are 281 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94850
-----------------------------------------
Data retrieval for 94850 complete
Getting results for next zip code.. there are 280 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94803
-----------------------------------------
Data retrie

Now processing query set 1 of max 20 for 94966
-----------------------------------------
Data retrieval for 94966 complete
Getting results for next zip code.. there are 246 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94920
-----------------------------------------
Data retrieval for 94920 complete
Getting results for next zip code.. there are 245 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94503
-----------------------------------------
Data retrieval for 94503 complete
Getting results for next zip code.. there are 244 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94589
-----------------------------------------
Data retrieval for 94589 complete
Getting results for next zip code.. there are 243 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94515
-----------------------------------------
Data retrie

...

Now processing query set 1 of max 20 for 94010
-----------------------------------------
Data retrieval for 94010 complete
Getting results for next zip code.. there are 172 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94025
-----------------------------------------
Data retrieval for 94025 complete
Getting results for next zip code.. there are 171 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94026
-----------------------------------------
Data retrieval for 94026 complete
Getting results for next zip code.. there are 170 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94027
-----------------------------------------
Data retrieval for 94027 complete
Getting results for next zip code.. there are 169 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94028
-----------------------------------------
Data retrie

Now processing query set 1 of max 20 for 95031
-----------------------------------------
Data retrieval for 95031 complete
Getting results for next zip code.. there are 135 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95032
-----------------------------------------
Data retrieval for 95032 complete
Getting results for next zip code.. there are 134 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95033
-----------------------------------------
Data retrieval for 95033 complete
Getting results for next zip code.. there are 133 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95035
-----------------------------------------
Data retrieval for 95035 complete
Getting results for next zip code.. there are 132 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95036
-----------------------------------------
Data retrie

Now processing query set 1 of max 20 for 95121
-----------------------------------------
Data retrieval for 95121 complete
Getting results for next zip code.. there are 98 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95122
-----------------------------------------
Data retrieval for 95122 complete
Getting results for next zip code.. there are 97 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95123
-----------------------------------------
Data retrieval for 95123 complete
Getting results for next zip code.. there are 96 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95124
-----------------------------------------
Data retrieval for 95124 complete
Getting results for next zip code.. there are 95 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95125
-----------------------------------------
Data retrieval 

Now processing query set 1 of max 20 for 95190
-----------------------------------------
Data retrieval for 95190 complete
Getting results for next zip code.. there are 61 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95191
-----------------------------------------
Data retrieval for 95191 complete
Getting results for next zip code.. there are 60 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95192
-----------------------------------------
Data retrieval for 95192 complete
Getting results for next zip code.. there are 59 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95193
-----------------------------------------
Data retrieval for 95193 complete
Getting results for next zip code.. there are 58 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95196
-----------------------------------------
Data retrieval 

Now processing query set 1 of max 20 for 94927
-----------------------------------------
Data retrieval for 94927 complete
Getting results for next zip code.. there are 24 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94928
-----------------------------------------
Data retrieval for 94928 complete
Getting results for next zip code.. there are 23 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94931
-----------------------------------------
Data retrieval for 94931 complete
Getting results for next zip code.. there are 22 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 95448
-----------------------------------------
Data retrieval for 95448 complete
Getting results for next zip code.. there are 21 zips left
-----------------------------------------
Now processing query set 1 of max 20 for 94952
-----------------------------------------
Data retrieval 

In [16]:
# Show cities_df with updated total # of restaurants
zip_df.head()

,Zip,City,County,Total # of Restaurants
0,94501,Alameda,Alameda,7800
1,94502,Alameda,Alameda,342
2,94706,Albany,Alameda,719
3,94707,Albany,Alameda,635
4,94710,Albany,Alameda,2300


In [18]:
zip_df.to_csv("Resources/total_count_zipcodes.csv")